In [ ]:
from load import *
from torch import Tensor, nn
import torch
from modules import *

ROOT = "./Datas/Week 8"


Q = 1.60217663e-19

def init_weights(m):
    try:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
    except AttributeError:
        pass

def load():
    ep = load_elec_potential().reshape(-1, 2193)
    sc = (load_space_charge() * (-Q)).reshape(-1, 2193)
    epsc = torch.cat([ep, sc], axis = -1)
    return epsc